<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/seed(s%2Cn%2Ca%2Ca%2Ce)_with_CGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load EEG data
data = pd.read_csv("/content/labeled_data(sad,neu,anx,ang,exc).csv")  # Update path if needed

# Extract features (X) and labels (y)
X = data.drop('label', axis=1).values
y = data['label'].values

# Label encoding (convert emotions to integers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (40728, 13), Test shape: (10182, 13)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape the input data for LSTM
# Assuming each sample has a single time step and 11 features:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])  # Reshape to (40728, 1, 13)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])    # Reshape to (10182, 1, 13)
# Define LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])), # Now input_shape is (1, 11)
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train on real data only
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
_, accuracy = model.evaluate(X_test, y_test)
print(f"Baseline Test Accuracy (Real Data Only): {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7416 - loss: 0.7638 - val_accuracy: 0.8838 - val_loss: 0.2809
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9062 - loss: 0.2428 - val_accuracy: 0.8922 - val_loss: 0.2649
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9177 - loss: 0.2087 - val_accuracy: 0.9235 - val_loss: 0.1874
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9170 - loss: 0.1999 - val_accuracy: 0.9273 - val_loss: 0.1810
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9296 - loss: 0.1705 - val_accuracy: 0.9388 - val_loss: 0.1553
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9302 - loss: 0.1628 - val_accuracy: 0.9396 - val_loss: 0.1617
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9385 - loss: 0.1462 - val_accuracy: 0.9503 - val_loss: 0.1176
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9430 - loss: 0.1297 - val_accuracy: 0.

In [7]:

from tensorflow.keras.layers import Input, Dense, Conv1DTranspose, LeakyReLU, Flatten, Embedding, Concatenate, Reshape
from tensorflow.keras.models import Model, Sequential

latent_dim = 100
n_features = X_train.shape[2]
time_steps = X_train.shape[1]
n_classes = len(np.unique(y_encoded))

# Generator
def build_generator():
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(1,))
    label_embedding = Flatten()(Embedding(n_classes, latent_dim)(label_input))

    combined = tf.keras.layers.multiply([noise_input, label_embedding])

    x = Dense(128 * time_steps, activation="relu")(combined)
    x = Reshape((time_steps, 128))(x)
    x = Conv1DTranspose(64, kernel_size=3, padding="same", activation="relu")(x)
    x = Conv1DTranspose(n_features, kernel_size=3, padding="same")(x)
    output = Reshape((time_steps, n_features))(x)

    return Model([noise_input, label_input], output)

# Discriminator
def build_discriminator():
    eeg_input = Input(shape=(time_steps, n_features))
    label_input = Input(shape=(1,))
    label_embedding = Flatten()(Embedding(n_classes, time_steps * n_features)(label_input))
    label_embedding = Reshape((time_steps, n_features))(label_embedding)

    combined = Concatenate()([eeg_input, label_embedding])

    x = Flatten()(combined)
    x = Dense(128, activation=LeakyReLU(0.2))(x)
    x = Dense(1, activation='sigmoid')(x)

    return Model([eeg_input, label_input], x)


In [8]:
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

z = Input(shape=(latent_dim,))
label = Input(shape=(1,))
fake_eeg = generator([z, label])
discriminator.trainable = False
validity = discriminator([fake_eeg, label])

combined = Model([z, label], validity)
combined.compile(loss='binary_crossentropy', optimizer='adam')


In [9]:
import random
import numpy as np # Importing numpy

def train_gan(epochs=5000, batch_size=64):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_eegs = X_train[idx]
        real_labels = y_train[idx]

        # The real_eegs already have the correct shape (half_batch, 1, n_features)
        # because X_train was reshaped earlier. Remove the reshape line:
        # real_eegs = real_eegs.reshape(real_eegs.shape[0], 1, real_eegs.shape[1])

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_eegs = generator.predict([noise, real_labels])

        d_loss_real = discriminator.train_on_batch([real_eegs, real_labels], np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch([fake_eegs, real_labels], np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        sampled_labels = np.random.randint(0, n_classes, batch_size)
        valid_y = np.ones((batch_size, 1))

        g_loss = combined.train_on_batch([noise, sampled_labels], valid_y)

        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}%] [G loss: {g_loss}]")

train_gan()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

In [15]:
def generate_synthetic_data(n_samples=10000):
    noise = np.random.normal(0, 1, (n_samples, latent_dim))
    labels = np.random.randint(0, n_classes, n_samples)
    synthetic_eeg = generator.predict([noise, labels])
    return synthetic_eeg, labels

synthetic_X, synthetic_y = generate_synthetic_data(10000)

# Combine Real + Synthetic Data
X_augmented = np.vstack([X_train, synthetic_X])
y_augmented = np.concatenate([y_train, synthetic_y])

print(f"Augmented Data Shape: {X_augmented.shape}, Labels Shape: {y_augmented.shape}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Augmented Data Shape: (50728, 1, 13), Labels Shape: (50728,)


In [17]:
model_augmented = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(n_classes, activation='softmax')
])

model_augmented.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train with Augmented Data
history_aug = model_augmented.fit(X_augmented, y_augmented, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
_, accuracy_aug = model_augmented.evaluate(X_test, y_test)
print(f"Augmented Test Accuracy (Real + Synthetic): {accuracy_aug * 100:.2f}%")


Epoch 1/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6462 - loss: 0.8958 - val_accuracy: 0.8907 - val_loss: 0.2761
Epoch 2/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7718 - loss: 0.5050 - val_accuracy: 0.9104 - val_loss: 0.2198
Epoch 3/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7795 - loss: 0.4792 - val_accuracy: 0.9248 - val_loss: 0.1884
Epoch 4/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7835 - loss: 0.4669 - val_accuracy: 0.9323 - val_loss: 0.1663
Epoch 5/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7889 - loss: 0.4517 - val_accuracy: 0.9308 - val_loss: 0.1662
Epoch 6/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7881 - loss: 0.4471 - val_accuracy: 0.9394 - val_loss: 0.1509
Epoch 7/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7958 - loss: 0.4336 - val_accuracy: 0.9499 - val_loss: 0.1277
Epoch 8/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7975 - loss: 0.4214 - val_accuracy: 0.

In [18]:
generator.save('/content/generator_model.h5')
print("✅ Generator saved successfully!")


✅ Generator saved successfully!


In [19]:
discriminator.save('/content/discriminator_model.h5')
print("✅ Discriminator saved successfully!")


✅ Discriminator saved successfully!


In [21]:
model_augmented.save('/content/lstm_model.h5')
print("✅ LSTM classifier saved successfully!")

✅ LSTM classifier saved successfully!
